# DataFetcher Testing Notebook

This notebook demonstrates the usage of the DataFetcher class for fetching stock market data.

In [5]:
import asyncio
from datetime import datetime, timedelta
import pandas as pd
import logging
from data_fetcher import DataFetcher

# Set up logging
logging.basicConfig(level=logging.INFO)

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [6]:
import os
from dotenv import load_dotenv

# Configuration
load_dotenv()

API_KEY = os.getenv('POLYGON_API_KEY')
TEST_SYMBOLS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']
BATCH_SIZE = 2  # Smaller batch size for testing

In [8]:
# Initialize DataFetcher
fetcher = DataFetcher(API_KEY, TEST_SYMBOLS, BATCH_SIZE)

# Test single symbol fetch
async def test_single_fetch():
    start_time = datetime.now() - timedelta(days=1)
    data = await fetcher.fetch_minute_stock_data('AAPL', start_time)
    return data

# Run single fetch test
single_symbol_data = await test_single_fetch()
print(f"Fetched {len(single_symbol_data)} records for AAPL")
if single_symbol_data:
    print("Sample data point:", single_symbol_data[0])

Fetched 819 records for AAPL
Sample data point: MarketData(timestamp=datetime.datetime(2024, 11, 1, 1, 0), symbol='AAPL', price=222.9, volume=4558)


In [ ]:
# Test batch fetch
async def test_batch_fetch():
    start_time = datetime.now() - timedelta(days=1)
    data = await fetcher.fetch_stock_batch(TEST_SYMBOLS[:2], start_time)
    return data

batch_data = await test_batch_fetch()
print(f"Fetched {len(batch_data)} total records for batch")
# Group by symbol and count
if batch_data:
    df = pd.DataFrame([(d.symbol, d.timestamp) for d in batch_data], 
                      columns=['symbol', 'timestamp'])
    print("\nRecords per symbol:")
    print(df.groupby('symbol').count())

In [ ]:
# Test full fetch
async def test_full_fetch():
    start_time = datetime.now() - timedelta(days=1)
    df = await fetcher.fetch_stock_all(start_time)
    return df

full_data_df = await test_full_fetch()

if not full_data_df.empty:
    print("\nData summary:")
    print(full_data_df.groupby('symbol').agg({
        'price': ['count', 'mean', 'std'],
        'volume': 'sum'
    }))

In [ ]:
# Test cache functionality
from data_fetcher import MarketData

test_data = MarketData(
    timestamp=datetime.now(),
    symbol='AAPL',
    price=150.0,
    volume=1000
)

fetcher.update_cache(test_data)
print("Cached data:", fetcher._cache['AAPL'])